# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 27 2022 10:36AM,240443,16,EMERSA-8188293,"Emersa Waterbox, LLC",Released
1,Jun 27 2022 10:31AM,240513,10,PRF-32207,Bio-PRF,Released
2,Jun 27 2022 10:31AM,240513,10,PRF-32208,Bio-PRF,Released
3,Jun 27 2022 10:31AM,240513,10,PRF-32217,Bio-PRF,Released
4,Jun 27 2022 10:31AM,240513,10,PRF-32223,Bio-PRF,Released
5,Jun 27 2022 10:31AM,240513,10,PRF-32227,Bio-PRF,Released
6,Jun 27 2022 10:31AM,240513,10,PRF-32230,Bio-PRF,Released
7,Jun 27 2022 10:31AM,240513,10,PRF-32271,Bio-PRF,Released
8,Jun 27 2022 10:31AM,240513,10,PRF-32274,Bio-PRF,Released
9,Jun 27 2022 10:31AM,240513,10,PRF-32280,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,240508,Released,10
42,240509,Released,18
43,240511,Released,1
44,240512,Released,12
45,240513,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240508,NaN,10.0
240509,NaN,18.0
240511,NaN,1.0
240512,NaN,12.0
240513,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240443,0.0,1.0
240455,0.0,4.0
240463,35.0,12.0
240464,0.0,1.0
240466,16.0,42.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240443,0,1
240455,0,4
240463,35,12
240464,0,1
240466,16,42


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240443,0,1
1,240455,0,4
2,240463,35,12
3,240464,0,1
4,240466,16,42


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240443,,1
1,240455,,4
2,240463,35,12
3,240464,,1
4,240466,16,42


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 27 2022 10:36AM,240443,16,"Emersa Waterbox, LLC"
1,Jun 27 2022 10:31AM,240513,10,Bio-PRF
12,Jun 27 2022 10:23AM,240512,10,"Methapharm, Inc."
24,Jun 27 2022 10:18AM,240495,10,ISDIN Corporation
40,Jun 27 2022 10:18AM,240511,20,Sartorius Stedim North America
41,Jun 27 2022 10:13AM,240507,15,"Mizner Bioscience, LLC"
42,Jun 27 2022 10:12AM,240506,15,"Carwin Pharmaceutical Associates, LLC"
46,Jun 27 2022 10:10AM,240504,15,"Person & Covey, Inc."
56,Jun 27 2022 10:10AM,240509,10,Eywa Pharma Inc.
74,Jun 27 2022 10:07AM,240508,15,"Laser Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 27 2022 10:36AM,240443,16,"Emersa Waterbox, LLC",,1
1,Jun 27 2022 10:31AM,240513,10,Bio-PRF,,11
2,Jun 27 2022 10:23AM,240512,10,"Methapharm, Inc.",,12
3,Jun 27 2022 10:18AM,240495,10,ISDIN Corporation,,16
4,Jun 27 2022 10:18AM,240511,20,Sartorius Stedim North America,,1
5,Jun 27 2022 10:13AM,240507,15,"Mizner Bioscience, LLC",,1
6,Jun 27 2022 10:12AM,240506,15,"Carwin Pharmaceutical Associates, LLC",,4
7,Jun 27 2022 10:10AM,240504,15,"Person & Covey, Inc.",,10
8,Jun 27 2022 10:10AM,240509,10,Eywa Pharma Inc.,,18
9,Jun 27 2022 10:07AM,240508,15,"Laser Pharmaceuticals, LLC",,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 10:36AM,240443,16,"Emersa Waterbox, LLC",1,
1,Jun 27 2022 10:31AM,240513,10,Bio-PRF,11,
2,Jun 27 2022 10:23AM,240512,10,"Methapharm, Inc.",12,
3,Jun 27 2022 10:18AM,240495,10,ISDIN Corporation,16,
4,Jun 27 2022 10:18AM,240511,20,Sartorius Stedim North America,1,
5,Jun 27 2022 10:13AM,240507,15,"Mizner Bioscience, LLC",1,
6,Jun 27 2022 10:12AM,240506,15,"Carwin Pharmaceutical Associates, LLC",4,
7,Jun 27 2022 10:10AM,240504,15,"Person & Covey, Inc.",10,
8,Jun 27 2022 10:10AM,240509,10,Eywa Pharma Inc.,18,
9,Jun 27 2022 10:07AM,240508,15,"Laser Pharmaceuticals, LLC",10,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 10:36AM,240443,16,"Emersa Waterbox, LLC",1,
1,Jun 27 2022 10:31AM,240513,10,Bio-PRF,11,
2,Jun 27 2022 10:23AM,240512,10,"Methapharm, Inc.",12,
3,Jun 27 2022 10:18AM,240495,10,ISDIN Corporation,16,
4,Jun 27 2022 10:18AM,240511,20,Sartorius Stedim North America,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 10:36AM,240443,16,"Emersa Waterbox, LLC",1,NaN
1,Jun 27 2022 10:31AM,240513,10,Bio-PRF,11,NaN
2,Jun 27 2022 10:23AM,240512,10,"Methapharm, Inc.",12,NaN
3,Jun 27 2022 10:18AM,240495,10,ISDIN Corporation,16,NaN
4,Jun 27 2022 10:18AM,240511,20,Sartorius Stedim North America,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 10:36AM,240443,16,"Emersa Waterbox, LLC",1,0.0
1,Jun 27 2022 10:31AM,240513,10,Bio-PRF,11,0.0
2,Jun 27 2022 10:23AM,240512,10,"Methapharm, Inc.",12,0.0
3,Jun 27 2022 10:18AM,240495,10,ISDIN Corporation,16,0.0
4,Jun 27 2022 10:18AM,240511,20,Sartorius Stedim North America,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4809691,494,16.0
12,721432,15,35.0
15,1443017,50,14.0
16,721391,3,0.0
19,480949,5,0.0
20,1202453,22,0.0
21,1202492,5,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4809691,494,16.0
1,12,721432,15,35.0
2,15,1443017,50,14.0
3,16,721391,3,0.0
4,19,480949,5,0.0
5,20,1202453,22,0.0
6,21,1202492,5,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,494,16.0
1,12,15,35.0
2,15,50,14.0
3,16,3,0.0
4,19,5,0.0
5,20,22,0.0
6,21,5,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,494.0
1,12,Released,15.0
2,15,Released,50.0
3,16,Released,3.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,16.0,35.0,14.0,0.0,0.0,0.0,0.0
Released,494.0,15.0,50.0,3.0,5.0,22.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,16.0,35.0,14.0,0.0,0.0,0.0,0.0
1,Released,494.0,15.0,50.0,3.0,5.0,22.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,16.0,35.0,14.0,0.0,0.0,0.0,0.0
1,Released,494.0,15.0,50.0,3.0,5.0,22.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()